# Housing Market Prediction
authors: Scott Breitbach, Pushkar Chougule & Rachel Nelson

class: DSC630-T301 Predictive Analytics (2221-1)




In [220]:
import requests
from bs4 import BeautifulSoup
import json
import time
import csv
import pandas as pd

In [221]:
class ZillowScraper():
    results = []
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'cookie': 'zguid=23|%24d14f0059-d03b-4422-9f57-5862fcd13490; _ga=GA1.2.1741285320.1590755697; zjs_user_id=null; zjs_anonymous_id=%22d14f0059-d03b-4422-9f57-5862fcd13490%22; __gads=ID=1050523ba93d593d:T=1590755700:S=ALNI_MZlJJ_xqSbd51oJisV_HY4g017Ehw; _gcl_au=1.1.2000298647.1590755705; KruxPixel=true; _fbp=fb.1.1590755705919.1815197270; _pxvid=d6c5ec75-a1a8-11ea-b8a9-0242ac120009; KruxAddition=true; JSESSIONID=3E7EBDB1F8931DF7D0DE9992546AE0B3; zgsession=1|200e23e0-9534-4d27-931f-caa3de6b483b; _gid=GA1.2.1328942480.1590858452; _gat=1; DoubleClickSession=true; GASession=true; _uetsid=fdde22d5-862a-8a7d-93e4-a16c574edf91; _pin_unauth=YzUyOGQ2OGMtMmQ3YS00NGZkLTg3MmEtOGJlODM1YWMwMTA1; _px3=026336d3721eec42bcdec3278ad2d3ac2014d5e65707b21624fb2e743d9a89be:mq3WRz2RNL5PBIvbYNHCxq5VfXHXy2YKC+8Lqn97pIw8MiKppH7Cx7AjKzbAFi1zcehKGY36aIgsnE9NiPKwlw==:1000:4U1o3ogIQ0KzfyMd2QYEFGDnD1augezy5bJlzEn9ZHE89B2uEIxDg8BmsGj8szPwyIz1Yv15S2V0TV5P+0jCFisfGk92XM4DM7K13GCtNr0HXhNGftVBFxVrCv8ApRphw/Qwj7AcagCh9i6FPiQGLFruxVASJXLsNpFeWimekVY=; AWSALB=ZKAGBcH2BwM6D1bRKOPynbOqyclySGz5U/fZB+wO3MYQ91UR9A5rFVtFsmjOkrMASUJguhtsJRZDM7IlBiWVT/pGw2S0BjxgEZmpFPrBZEqU2lWTE2NMArtecZD2; AWSALBCORS=ZKAGBcH2BwM6D1bRKOPynbOqyclySGz5U/fZB+wO3MYQ91UR9A5rFVtFsmjOkrMASUJguhtsJRZDM7IlBiWVT/pGw2S0BjxgEZmpFPrBZEqU2lWTE2NMArtecZD2; search=6|1593450465587%7Crect%3D40.843698984643765%252C-73.50417109960938%252C40.567821651427245%252C-74.45174190039063%26rid%3D6181%26disp%3Dmap%26mdm%3Dauto%26p%3D2%26z%3D0%26lt%3Dfsbo%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%096181%09%09%09%09%09%09',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }

    def fetch(self, url, params):
        response = requests.get(url, headers=self.headers, params=params)
        print(response.status_code)
        return response

    def parse(self, response):
        content = BeautifulSoup(response, 'lxml')
        deck = content.find('ul', {'class': 'photo-cards photo-cards_wow photo-cards_short'})
        for card in deck.contents:
            script = card.find('script', {'type': 'application/ld+json'})
            if script:
                script_json = json.loads(script.contents[0])

                self.results.append({
                    'latitude': script_json['geo']['latitude'],
                    'longitude': script_json['geo']['longitude'],
                    'floorSize': script_json['floorSize']['value'],
                    'url': script_json['url'],
                    'price': card.find('div', {'class': 'list-card-price'}).text,
                    'details': card.find('ul',{'class': 'list-card-details'}),

                })

    def to_csv(self):
        with open('zillow.csv', 'w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys())
            writer.writeheader()

            for row in self.results:
                writer.writerow(row)

    def run(self):
        url = 'https://www.zillow.com/seattle-wa/houses/' #https://www.zillow.com/new-york-ny/fsbo/

        for page in range(1, 13):
            params = {
                'searchQueryState': '{"pagination":{"currentPage": %s},"mapBounds":{"west":-74.40093013281245,"east":-73.55498286718745,"south":40.4487909557045,"north":40.96202658306895},"regionSelection":[{"regionId":6181,"regionType":6}],"isMapVisible":false,"filterState":{"isForSaleByAgent":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false}},"isListVisible":true}' %page
            }
            res = self.fetch(url, params)
            self.parse(res.text)
            time.sleep(2)
        self.to_csv()


if __name__ == '__main__':
    scraper = ZillowScraper()
    scraper.run()

200
200
200
200
200
200
200
200
200
200
200
200


In [222]:
df = pd.read_csv("zillow.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   42 non-null     float64
 1   longitude  42 non-null     float64
 2   floorSize  42 non-null     object 
 3   url        42 non-null     object 
 4   price      42 non-null     object 
 5   details    42 non-null     object 
dtypes: float64(2), object(4)
memory usage: 1.4+ KB


In [223]:
df = df.replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
df = df.replace('<abbr class="list-card-label"> <!-- -->', ' ', regex=True)
df = df.replace('</abbr></li><li class="">', ' ', regex=True)
df = df.replace('<abbr class="list-card-label"> <!-- -->', ' ', regex=True)
df = df.replace('</abbr></li><li class="list-card-statusText">', ' ', regex=True)
df = df.replace('</li></ul>', ' ', regex=True)
df = df.replace('</li><li class="">', ' ', regex=True)
df = df.replace('Studio', '1 Studio ', regex=True)
df = df.replace('- ', '', regex=True)


In [227]:
# new data frame with split value columns
# new data frame with split value columns
new = df["details"].str.split(" ", n = 7, expand = True)

# making separate last name column from new data frame
df["beds"]= new[1]
df["baths"]= new[3]
df["sqft"]= new[5]
df["forsaleby"]= new[7]


In [228]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["price",'beds','baths','sqft'], inplace=True)

In [226]:
df.head(5)

,latitude,longitude,floorSize,url,price,details,beds,baths,sqft,forsaleby
0,40.801077,-73.947794,830,https://www.zillow.com/homedetails/40-W-116th-...,"$995,000",2 bds 1 ba 830 sqft For sale by owner,2,1,830,For sale by owner
1,40.768959,-73.956041,800,https://www.zillow.com/homedetails/345-E-73rd-...,"$795,000",2 bds 1 ba 800 sqft For sale by owner,2,1,800,For sale by owner
2,40.707698,-74.009803,"1,387",https://www.zillow.com/homedetails/20-Pine-St-...,"$1,568,000","2 bds 2 ba 1,387 sqft For sale by owner",2,2,"1,387",For sale by owner
3,40.766042,-73.952338,477,https://www.zillow.com/homedetails/524-E-72nd-...,"$775,000",1 bd 1 ba 477 sqft For sale by owner,1,1,477,For sale by owner
5,40.766900,-73.928955,630,https://www.zillow.com/homedetails/2117-31st-A...,"$746,000",1 bd 1 ba 630 sqft For sale by owner,1,1,630,For sale by owner
7,40.684817,-73.966614,625,https://www.zillow.com/homedetails/451-Clinton...,"$675,000",1 bd 1 ba 625 sqft For sale by owner,1,1,625,For sale by owner
8,40.773471,-73.952304,590,https://www.zillow.com/homedetails/345-E-80th-...,"$800,000",1 bd 1 ba 590 sqft For sale by owner,1,1,590,For sale by owner
9,47.719222,-122.314786,"1,620",https://www.zillow.com/homedetails/1211-NE-125...,"$1,000,000","4 bds 3 ba 1,620 sqft For sale by owner",4,3,"1,620",For sale by owner
10,47.613218,-122.298036,"1,215",https://www.zillow.com/homedetails/2616-E-Unio...,"$950,000","3 bds 2 ba 1,215 sqft For sale by owner",3,2,"1,215",For sale by owner
11,47.612565,-122.338884,"1,607",https://www.zillow.com/homedetails/1920-4th-Av...,"$1,450,000","2 bds 2 ba 1,607 sqft For sale by owner",2,2,"1,607",For sale by owner
